In [ ]:
!pip install transformers datasets accelerate peft bitsandbytes torch

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType

In [ ]:
# Nome do modelo
MODEL_NAME = "tiiuae/falcon-7b"

In [ ]:
# Carregar o tokenizador
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME) 

In [ ]:
tokenizer.pad_token = tokenizer.eos_token # Usar o token de EOS para padding

In [ ]:

# Carregar o modelo com quantização para economizar memória
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,  # Usa menos VRAM
    device_map="auto"  # Distribui automaticamente entre CPU e GPU
)

In [ ]:
# Dados de treino
data = [
    {"input": "O que é Golang?", "output": "Golang, ou Go, é uma linguagem de programação criada pelo Google em 2007, focada em desempenho, concorrência e simplicidade."},
    {"input": "Quais são as principais características do Golang?", "output": "Golang é conhecido por sua sintaxe simples, garbage collection eficiente, suporte nativo à concorrência e compilação rápida."},
    {"input": "O que são goroutines em Golang?", "output": "Goroutines são funções ou métodos executados de forma concorrente em Go, sendo mais leves que threads do sistema operacional."},
    {"input": "Como criar uma goroutine em Go?", "output": "Basta usar a palavra-chave 'go' antes de chamar uma função, como em 'go minhaFuncao()'."},
    {"input": "O que é um canal (channel) em Golang?", "output": "Canais são estruturas de comunicação entre goroutines, permitindo a troca segura de dados concorrentes."},
    {"input": "Qual é a diferença entre um buffer e um canal sem buffer em Golang?", "output": "Canais sem buffer bloqueiam o envio até que haja um receptor, enquanto canais com buffer armazenam mensagens até o limite do buffer."},
    {"input": "Como declarar uma variável em Golang?", "output": "Use 'var nome tipo' ou ':=' para inferência de tipo, como 'var idade int' ou 'idade := 25'."},
    {"input": "Golang é uma linguagem compilada ou interpretada?", "output": "Golang é uma linguagem compilada, gerando binários executáveis sem necessidade de uma máquina virtual."},
    {"input": "O que são interfaces em Golang?", "output": "Interfaces definem um conjunto de métodos que um tipo deve implementar, permitindo polimorfismo sem herança."},
    {"input": "Como funciona o gerenciamento de memória em Golang?", "output": "Go possui um garbage collector que gerencia automaticamente a alocação e desalocação de memória."},
    {"input": "O que é um struct em Golang?", "output": "Structs são tipos compostos que agrupam múltiplos campos de dados, semelhantes a classes, mas sem herança."},
    {"input": "Como definir um struct em Go?", "output": "Use a palavra-chave 'type' seguida do nome e dos campos, como 'type Pessoa struct { Nome string; Idade int }'."},
    {"input": "Go suporta orientação a objetos?", "output": "Go não possui classes nem herança, mas suporta composição via structs e interfaces."},
    {"input": "Como tratar erros em Golang?", "output": "Go usa a abordagem explícita de erros, retornando valores 'error' em funções e verificando com 'if err != nil'."},
    {"input": "Como declarar e utilizar um array em Go?", "output": "Arrays são declarados com 'var a [5]int' e têm tamanho fixo, sendo acessados via índices."},
    {"input": "O que são slices em Golang?", "output": "Slices são abstrações flexíveis sobre arrays, permitindo redimensionamento dinâmico e melhor gerenciamento de memória."},
    {"input": "Qual a diferença entre array e slice em Go?", "output": "Arrays têm tamanho fixo, enquanto slices são referências a arrays e podem crescer dinamicamente."},
    {"input": "Como inicializar um map em Go?", "output": "Use 'make(map[string]int)' ou 'map[string]int{'chave': 10}' para criar um dicionário de chave-valor."},
    {"input": "Go tem suporte para exceções?", "output": "Go não possui exceções como Java ou Python, mas usa 'panic' e 'recover' para capturar falhas críticas."},
    {"input": "Como funciona defer em Go?", "output": "O 'defer' adia a execução de uma função até o final do escopo atual, útil para liberar recursos como arquivos."},
    {"input": "O que é o pacote fmt em Go?", "output": "O pacote 'fmt' fornece funções para formatação e impressão de strings, como 'fmt.Println()'."},
    {"input": "O que é reflection em Golang?", "output": "Reflection permite inspecionar e modificar tipos e valores em tempo de execução, usando o pacote 'reflect'."},
    {"input": "O que é um ponteiro em Go?", "output": "Ponteiros armazenam endereços de memória, permitindo modificar valores sem cópia."},
    {"input": "Como criar um ponteiro em Go?", "output": "Use '&' para obter o endereço de uma variável e '*' para acessar o valor, como 'var p *int = &x'."},
    {"input": "Go possui genéricos?", "output": "Desde a versão Go 1.18, a linguagem suporta genéricos, permitindo funções e estruturas de dados flexíveis sem perda de tipo."},
    {"input": "Como instalar pacotes externos em Go?", "output": "Use 'go get' seguido da URL do pacote, como 'go get github.com/gorilla/mux'."},
    {"input": "Como definir uma constante em Go?", "output": "Use a palavra-chave 'const', como em 'const Pi = 3.14'."},
    {"input": "Como criar um servidor HTTP em Go?", "output": "Use o pacote 'net/http' e a função 'http.ListenAndServe()' para criar um servidor web simples."},
    {"input": "Quais são os comandos básicos do Go Modules?", "output": "'go mod init' cria um módulo, 'go mod tidy' limpa dependências e 'go list -m all' lista os pacotes instalados."}
]


In [ ]:
# Converter para Dataset Hugging Face
dataset = Dataset.from_list(data)

In [ ]:
# Tokenizar e criar labels
def tokenize_function(examples):
    prompt = "Pergunta: " + examples["input"] + "\nResposta: " + examples["output"]

    # Tokenizar entrada e saída juntas
    tokens = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)

    # Criar labels: rótulos são os mesmos input_ids, mas ignoramos o padding (-100)
    tokens["labels"] = tokens["input_ids"].copy()
    tokens["labels"] = [
        -100 if token == tokenizer.pad_token_id else token for token in tokens["labels"]
    ]

    return tokens


In [ ]:
# Tokenizar o dataset
dataset = dataset.map(tokenize_function) 

In [ ]:
# Configurar LoRA
lora_config = LoraConfig(
    r=8,  # Define o tamanho das matrizes auxiliares LoRA
    lora_alpha=32,  # Define a escala do ajuste LoRA
    lora_dropout=0.05,  # Adiciona dropout para evitar overfitting
    bias="none", # Remove o bias para economizar memória
    task_type=TaskType.CAUSAL_LM  # Define o modelo como um "causal language model"
)

In [ ]:
# Aplicar LoRA ao Falcon 7B
model = get_peft_model(model, lora_config)

In [ ]:
# Exibir os parâmetros treináveis do modelo
model.print_trainable_parameters()

In [ ]:
# Configurar os hiperparâmetros do treinamento
training_args = TrainingArguments(
    output_dir="./falcon-7b-lora-finetuned",  # Onde salvar o modelo treinado
    per_device_train_batch_size=2,  # Usa batch pequeno para economizar VRAM
    gradient_accumulation_steps=4,  # Simula batch maior sem estourar a VRAM
    num_train_epochs=3,  # Número de épocas de treinamento
    learning_rate=2e-5,  # Taxa de aprendizado otimizada para LoRA (0.00002)
    logging_dir="./logs",  # Diretório de logs para análise
    logging_steps=10,  # Salvar logs a cada 10 steps
    save_strategy="epoch",  # Salvar checkpoints no final de cada época
    fp16=True,  # Usa FP16 para reduzir o consumo de VRAM
    push_to_hub=False,  # Se quiser salvar no Hugging Face, mude para True
    report_to="none"  # 🚀 Isso desativa o W&B corretamente!
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

# Iniciar o treinamento
trainer.train()

In [ ]:
# Salvar modelo treinado
model.save_pretrained("./falcon-7b-lora-finetuned") # Salva o modelo
tokenizer.save_pretrained("./falcon-7b-lora-finetuned") # Salva o tokenizador

In [ ]:
input_text = "O que é Golang?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda") # Tokenizar e mover para GPU

In [ ]:
# Gerar resposta com o modelo treinado
output = model.generate(
    input_ids, # Entrada
    attention_mask=input_ids.ne(tokenizer.pad_token_id), # Ignorar padding
    max_length=50, # Tamanho máximo da resposta
    temperature=0.5, # Controla a aleatoriedade da resposta
    top_p=0.9, # Controla a diversidade da resposta
    repetition_penalty=1.2,  # Penaliza palavras repetidas
    do_sample=True # Habilita a amostragem
)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True) # Decodifica e remove tokens especiais

print("\n🔹 Resposta Gerada:\n", generated_text)